# Это файл с решением задания №1 

### Вопрос:
Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) 


### Логика действий

Те, кто сдавали экзамены:
1. Посчитаем кол-во студентов, кто сдавал экзамены и сдавал их в двух разных семестрах    
Курс = семестр1 + семестр2  
2. Смотрим на каких предметах у нас есть экзамены (из интереса)

Те, кто не сдавали экзамены:  
1. Разделим дата-сет на тех, кто сдавали тесты и на пересдачников 
2. Рассмотрим пересдачников. У них должно быть >= 3 семестров, поскольку в первом семестре они дропнулись, а во втором и третьем сдали курс
3. Рассмотрим не-пересдачников. Посмотрим на их количество тестов в каждом предмете. 
4. По каждому предмету объясним ситуацию, почему некоторые студенты сдавали разное кол-во тестов
5. Посчитаем кол-во семестров в каждом из предмете.
6. Посмотрим, сколько предметов взяли студенты (нам нужно, чтобы они взяли >2 предмета)
7. Считаем финальное количество ребят!

### Качаем дата-сеты 

In [5]:
#Основные либы
import pandas as pd 

#Качаем дата-сеты
assesments = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/assessments.csv')
courses = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/courses.csv')
student_assesment = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/studentAssessment.csv')
student_registration = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/studentRegistration.csv')

#Связываем дата-сеты
courses_reg = pd.merge(courses,student_registration,on=['code_module', 'code_presentation'])
ocenki = pd.merge(courses_reg, assesments,on= ['code_module', 'code_presentation'])
full_ocenki = pd.merge(ocenki, student_assesment,on=['id_assessment', 'id_student'])

## Студенты, кто сдавали экзамен

Посмотрим на студентов, кто в принципе хорошо сдал экзамен: score >= 40

In [6]:
#Сабсет тех, у кого оценка > 40
full_ocenki.query('assessment_type == "Exam" & score == score & score >= 40').id_student.nunique()

4097

У нас всего 4097 студентов, кто хорошо сдали экзамены.   
Но сдача одного экзамена в рамках одного семестра ≠ сдача курса   
Сдача курса = сдача одного или двух экзаменов в рамках двух семестров!

Посмотрим на количество студентов, кто успешно сдал экзамен и закрыл только один семестр:

In [7]:
#Сабсет тех, кто успешно сдал только один семестр
full_ocenki.query('assessment_type == "Exam" & score == score & score >= 40') \
    .groupby('id_student', as_index = False) \
    .agg({'code_presentation' : 'count', 'assessment_type' : 'count'}).query('code_presentation == 1')

,id_student,code_presentation,assessment_type
0,23698,1,1
1,24213,1,1
2,27116,1,1
3,28046,1,1
4,28787,1,1
...,...,...,...
4092,2694886,1,1
4093,2694933,1,1
4094,2695608,1,1
4095,2697181,1,1


У нас всего 3802 студентов, кто сдавал экзамены в рамках одного семестра.   
Они сдали семестр, но не курс!   
Поэтому, мы их выкидываем!

А теперь давайте посмотрим на студентов, кто успешно сдали экзамен и закрыли 2 семестра:

In [8]:
#Сабсет тех, кто успешно закрыли 2 семестра
full_ocenki.query('assessment_type == "Exam" & score == score & score >= 40') \
    .groupby('id_student', as_index = False) \
    .agg({'code_presentation' : 'count', 'assessment_type' : 'count'}).query('code_presentation == 2')

,id_student,code_presentation,assessment_type
64,80329,2,2
66,80868,2,2
91,105523,2,2
108,123598,2,2
111,126394,2,2
...,...,...,...
3945,2498388,2,2
4016,2609908,2,2
4028,2622441,2,2
4038,2639310,2,2


А вот 295 студентов, кто успешно сдал курс  
Ведь они сдали 2 семестра, а не один

In [11]:
#Покрывает ли один экзамен два семестра?
full_ocenki.query('assessment_type == "Exam" & score == score & score >= 40') \
    .groupby('id_student', as_index = False) \
    .agg({'code_presentation' : 'count', 'assessment_type' : 'count'}).query('code_presentation == 2 & assessment_type == 1')

,id_student,code_presentation,assessment_type


Все студенты, кто сдал 2 экзамена успешно, сдали их в двух различных семестрах.  

Получается, что ни один экзамен не покрывает одновременно 2 семестра. Только 1 

####  Еще интересно по каким предметам у нас проводятся экзы?

In [12]:
full_ocenki.query('assessment_type == "Exam" & score == score')['code_module'].value_counts()
#Только для этих предметов были экзамены!

DDD    3044
CCC    1915
Name: code_module, dtype: int64

---

## Те, кто не сдавали экзамен

Давайте для начала разделим дата-сет на два:
- там где нет передсачников 
- там где есть пересдачники   
(на всякий)

In [13]:
#Дата-сет, кто не сдавал экзамены 
examini = full_ocenki[full_ocenki['assessment_type'] == "Exam"].id_student.unique().tolist()

no_ex = full_ocenki[~full_ocenki.id_student.isin(examini)]


In [14]:
#Список критерий
peresdachi = no_ex[no_ex['is_banked'] == 1].id_student.unique().tolist()

#Дата-сет без передсач
no_peresdach = no_ex[~no_ex.id_student.isin(peresdachi)]

#Дата-сет с передсачами
peresdachniki = no_ex[no_ex.id_student.isin(peresdachi)]

####  Давайте сначала рассмотрим пересдачников! 

In [15]:
#Сабсет тех, кто успешно пересдал предметы:
peresdachniki.query('assessment_type != "Exam" & date_unregistration != date_unregistration & score == score') \
    .groupby(['code_presentation', 'code_module','id_student'], as_index = False) \
    .agg({'weight' : 'sum', 'assessment_type' : 'count'}) \
    .query('weight == 100')

,code_presentation,code_module,id_student,weight,assessment_type
11,2013B,BBB,1797614,100.0,11
12,2013B,BBB,2524127,100.0,11
20,2013B,FFF,334224,100.0,12
21,2013J,BBB,86291,100.0,11
26,2013J,BBB,305935,100.0,11
...,...,...,...,...,...
290,2014J,FFF,595676,100.0,6
296,2014J,FFF,626349,100.0,11
301,2014J,FFF,1948159,100.0,12
307,2014J,FFF,2681198,100.0,12


У нас 82 студентов, кто успешно пересдал предметы   
Но тут трудность. Они фактически сдали 2 семестра. Анализ может неправильно сказать, что они успешно завершили курс   
Ведь один семестр – они дропнули, а второй – обратно начали учится  
Нужно выбрать среди них тех студентов, кто прошел еще один семестр   
По сути получается, что их кол-во code_presentation должно быть >= 3

In [16]:
#Сабсет пересдачников, кто сдал успешно 3 семестра
peresdachniki.query('assessment_type != "Exam" & date_unregistration != date_unregistration & score == score') \
    .groupby(['code_presentation', 'code_module','id_student'], as_index = False) \
    .agg({'weight' : 'sum', 'assessment_type' : 'count','code_presentation' : pd.Series.nunique}) \
    .query('weight == 100 & code_presentation >= 3')

,code_module,id_student,weight,assessment_type,code_presentation


К сожалению, никто из из них не сдал полноценно один учебный курс :(((

Не берем их в анализ!

---

### Теперь рассмотрим не-пересдачников! 
Посчитаем количество тестов на каждом предмете

In [37]:
#Кол-во тестов на каждом предмете
no_peresdach.query('assessment_type != "Exam" & date_unregistration != date_unregistration & score == score') \
    .groupby(['code_presentation', 'code_module','id_student'], as_index = False) \
    .agg({'weight' : 'sum', 'assessment_type' : 'count'}) \
    .query('weight == 100').groupby('code_module')['assessment_type'].value_counts()

code_module  assessment_type
AAA          5                   547
BBB          11                 1757
             5                  1225
             4                     6
CCC          8                     8
DDD          6                    35
             13                    1
EEE          4                  1291
FFF          12                 3329
             11                   53
             10                   32
             5                    21
             9                    19
             8                    10
             6                     7
             7                     6
Name: assessment_type, dtype: int64

Что видим?
- в AAA точно 5 тестов 


- в BBB непонятно 5 или 11 или 4 


- в ССС точно 8 теста


- в DDD 6 или 13


- в ЕЕЕ 4 теста


- в FFF много...


- в GGG все очень круто! Там вообще нет весов для тестов :)))))))

### Выясним, почему в одних предметах неодинаковое кол-во тестов

#### Почему в предемете BBB 5 / 11 / 4 тестов (Предполагаю связано с семестрами)

In [19]:
#Сабсет студентов, кто имел предмет BBB, разделенный по семестрам; агрегация по кол-во тестов
no_peresdach.query('assessment_type != "Exam" & code_module == "BBB" & date_unregistration != date_unregistration & score == score') \
    .groupby(['code_module','code_presentation', 'is_banked', 'id_student'], as_index = False) \
    .agg({'weight' : 'sum','assessment_type': 'count'}) \
    .query('weight == 100') \
    .query('assessment_type == 5 | assessment_type == 11 | assessment_type == 4') \
    .groupby(['code_module','code_presentation'])['assessment_type'].value_counts()

code_module  code_presentation  assessment_type
BBB          2013B              11                  530
             2013J              11                  768
             2014B              11                  459
             2014J              5                  1225
                                4                     6
Name: assessment_type, dtype: int64

Cкорее всего получается такая история:   
- в семестрах: 2013B; 2013J; 2014B – препод давал студентам 11 тестиков 
- А в семестре: 2014J он решил, что 11 много и дал им по 5 тестов
- А что с чуваками, у кого 4 тестика, почему так?

Возьмем студентов у кого 5 тестиков 

In [20]:
#Два произвольных студента, у кого 5 тестов на предмете BBB 
no_peresdach.query('id_student == 26315 | id_student == 32930')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
17524,BBB,2014J,262,26315,-92.0,NaN,15020,TMA,19.0,0.0,5,0,1.0
17525,BBB,2014J,262,26315,-92.0,NaN,15021,TMA,54.0,10.0,39,0,60.0
17526,BBB,2014J,262,26315,-92.0,NaN,15022,TMA,110.0,20.0,109,0,72.0
17527,BBB,2014J,262,26315,-92.0,NaN,15023,TMA,152.0,35.0,151,0,62.0
17528,BBB,2014J,262,26315,-92.0,NaN,15024,TMA,201.0,35.0,201,0,62.0
17534,BBB,2014J,262,32930,-35.0,NaN,15020,TMA,19.0,0.0,19,0,1.0
17535,BBB,2014J,262,32930,-35.0,NaN,15021,TMA,54.0,10.0,60,0,57.0
17536,BBB,2014J,262,32930,-35.0,NaN,15022,TMA,110.0,20.0,110,0,59.0
17537,BBB,2014J,262,32930,-35.0,NaN,15023,TMA,152.0,35.0,159,0,56.0
17538,BBB,2014J,262,32930,-35.0,NaN,15024,TMA,201.0,35.0,201,0,52.0


Возьмем студентa с 4 тестами

In [21]:
#Произвольный студент с 4 тестами на предмете BBB
no_peresdach.query('id_student == 59272')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
17568,BBB,2014J,262,59272,-58.0,NaN,15021,TMA,54.0,10.0,47,0,64.0
17569,BBB,2014J,262,59272,-58.0,NaN,15022,TMA,110.0,20.0,106,0,66.0
17570,BBB,2014J,262,59272,-58.0,NaN,15023,TMA,152.0,35.0,149,0,63.0
17571,BBB,2014J,262,59272,-58.0,NaN,15024,TMA,201.0,35.0,199,0,72.0


Теперь понятно!

Походу у студентов у кого 5 тестов – им давали входной тестик, который весил 0 (weight == 0)

А студенты, кто имел 4 теста - не проходили входной тест 

Это нужно будет потом учитывать, потому тренировочные тестики нам не интересны!

#### Теперь рассмотрим DDD

In [22]:
#Сабсет студентов, кто имел предмет DDD, разделенный по семестрам; агрегация по кол-во тестов
no_peresdach.query('assessment_type != "Exam" & code_module == "DDD" & date_unregistration != date_unregistration & score == score') \
    .groupby(['code_module','code_presentation', 'id_student'], as_index = False) \
    .agg({'weight' : 'sum','assessment_type': 'count'}) \
    .query('weight == 100') \
    .query('assessment_type == 6 | assessment_type == 13') \
    .groupby(['code_module','code_presentation'])['assessment_type'].value_counts()

code_module  code_presentation  assessment_type
DDD          2013B              13                  1
             2013J              6                  13
             2014B              6                   7
             2014J              6                  15
Name: assessment_type, dtype: int64

Тоже самое:  
- 13 тестов только для одного семестра   
- а 6 для 3х остальных

#### Теперь рассмотрим FFF

Давайте посмотрим на студента с 12 тестами

In [19]:
#Студент с 12тью тестами
no_peresdach.query('id_student == 33915')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
136302,FFF,2013B,240,33915,-64.0,NaN,34865,CMA,222.0,0.0,37,0,93.0
136303,FFF,2013B,240,33915,-64.0,NaN,34866,CMA,222.0,0.0,60,0,90.0
136304,FFF,2013B,240,33915,-64.0,NaN,34867,CMA,222.0,0.0,95,0,86.0
136305,FFF,2013B,240,33915,-64.0,NaN,34868,CMA,222.0,0.0,136,0,87.0
136306,FFF,2013B,240,33915,-64.0,NaN,34869,CMA,222.0,0.0,202,0,89.0
136307,FFF,2013B,240,33915,-64.0,NaN,34871,CMA,222.0,0.0,213,0,83.0
136308,FFF,2013B,240,33915,-64.0,NaN,34870,CMA,222.0,0.0,213,0,97.0
136309,FFF,2013B,240,33915,-64.0,NaN,34860,TMA,19.0,12.5,17,0,86.0
136310,FFF,2013B,240,33915,-64.0,NaN,34861,TMA,47.0,12.5,46,0,94.0
136311,FFF,2013B,240,33915,-64.0,NaN,34862,TMA,89.0,25.0,87,0,93.0


У него 7 тестов весят 0 баллов.  
Может они тренировочные...?

Давате посмотрим на студентов без этих тренировочных тестов и посчитаем, сколько у них тестов! 

In [23]:
#Сабсет студентов, кто имел предмет FFF, без тренировочных тестов (weight != 0.0); разделенный по семестрам; агрегация по кол-во тестов
no_peresdach.query('assessment_type != "Exam" & code_module == "FFF" & weight != 0.0 & date_unregistration != date_unregistration & score == score') \
    .groupby(['code_module','code_presentation', 'id_student'], as_index = False) \
    .agg({'weight' : 'sum','assessment_type': 'count'}) \
    .query('weight == 100')  \
    .groupby(['code_module','code_presentation'])['assessment_type'].value_counts()

code_module  code_presentation  assessment_type
FFF          2013B              5                   819
             2013J              5                  1014
             2014B              5                   605
             2014J              5                  1039
Name: assessment_type, dtype: int64

Теперь все хорошо!   
У нас на предмет FFF 5 тестов, а любое другое количество – это сдача тренировочных тестов

#### Рассмотрим GGG

In [24]:
#Сабсет студентов, кто имел предмет GGG, разделенный по семестрам; агрегация по кол-во тестов
no_peresdach.query('assessment_type != "Exam" & code_module == "GGG" & date_unregistration != date_unregistration & score == score') \
    .groupby(['code_module','code_presentation', 'id_student'], as_index = False) \
    .agg({'assessment_type': 'count'}) \
    .groupby(['code_module','code_presentation'])['assessment_type'].value_counts()

code_module  code_presentation  assessment_type
GGG          2013J              9                  427
                                8                  113
                                7                  105
                                1                   49
                                2                   31
                                3                   27
                                6                   15
                                5                   12
                                4                    7
             2014B              9                  361
                                8                   87
                                7                   78
                                1                   48
                                2                   25
                                3                   23
                                4                   12
                                6                    8
                 

Тут вообще ничего не понятно, потому что не даны весы у тестов (weight == 0).   
Но, скорее всего общее количество тестов: 9   
Предмет, возможно, мягкий (как факультатив): тесты не обязательно проходить и предмет не имеет экзамена  
Короче правило такое: хотите этот предмет в зачетке – пройдите успешно 9 тестов    
В ином случае, вы просто прослушаете курс и плевать сдадите его или нет 

### Обобщая все вышесказанное: 

- в AAA точно 5 тестов 


- в BBB непонятно 4 или 11 тестов   
Это зависит от семестра!   
Там, где 4 теста у нас есть один входной тренировочный тестик (5 тестов)
 
 
- в ССС точно 8 теста


- в DDD 6 или 13 тестов   
Это зависит от семестра!


- в ЕЕЕ 4 теста


- в FFF точно 5 тестов и много много тренировочных


- в GGG скорее всего 9 тестов

### Давайте теперь подсчитаем количество семестров в каждом из предметов 

In [22]:
no_peresdach.groupby('code_module')['code_presentation'].nunique()

code_module
AAA    2
BBB    4
CCC    2
DDD    4
EEE    3
FFF    4
GGG    3
Name: code_presentation, dtype: int64

Окей, у нас все предметы охватывают два семестра >=2 семестра.   
Выберем студентов, у кого code_presentation == 2 (закончил 1 курс)

### А теперь давайте посмотрим, сколько предметов взяли студенты 

In [25]:
#Сначала был создан сабсет по студентам с агрегацией по сумме веса теста, уник. предметов и семестров, кол-во тестов 
#Второй сабсет – отбираем тех студентов, у кого сумма веса == 100 и кол-во семестров = 2 (закончил) курс
#Считаем value_counts по предметам, на оснве второго сабсета
no_peresdach.query('assessment_type != "Exam" & date_unregistration != date_unregistration & score == score') \
    .groupby('id_student', as_index = False).agg({'weight' : 'sum','code_module' : pd.Series.nunique, 
                                                        'assessment_type' : 'count',
                                                         'code_presentation' : pd.Series.nunique}) \
    .query('weight == 100 & code_presentation == 2')['code_module'].value_counts()

2    31
1     1
Name: code_module, dtype: int64

Так, у нас странная ситуация.   
У нас 1 чувак, без пересдач сдал тестики одного предмета в рамках двух семестров   
Давайте взглянем на него более подробно


In [26]:
#Выявляем этого студента
no_peresdach.query('assessment_type != "Exam" & date_unregistration != date_unregistration & score == score') \
    .groupby('id_student', as_index = False).agg({'weight' : 'sum','code_module' : pd.Series.nunique, 
                                                        'assessment_type' : 'count',
                                                         'code_presentation' : pd.Series.nunique}) \
    .query('weight == 100 & code_presentation == 2 & code_module == 1')

,id_student,weight,code_module,assessment_type,code_presentation
8750,605394,100.0,1,12,2


In [27]:
#Смотрим на его данные 
no_peresdach.query('id_student == 605394')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
117215,FFF,2013J,268,605394,-29.0,NaN,34873,TMA,19.0,12.5,33,0,74.0
125074,FFF,2014J,269,605394,-22.0,NaN,34904,CMA,241.0,0.0,215,0,67.0
125075,FFF,2014J,269,605394,-22.0,NaN,34905,CMA,241.0,0.0,219,0,73.0
125076,FFF,2014J,269,605394,-22.0,NaN,34906,CMA,241.0,0.0,237,0,67.0
125077,FFF,2014J,269,605394,-22.0,NaN,34907,CMA,241.0,0.0,229,0,62.0
125078,FFF,2014J,269,605394,-22.0,NaN,34908,CMA,241.0,0.0,238,0,53.0
125079,FFF,2014J,269,605394,-22.0,NaN,34910,CMA,241.0,0.0,238,0,69.0
125080,FFF,2014J,269,605394,-22.0,NaN,34909,CMA,241.0,0.0,243,0,52.0
125081,FFF,2014J,269,605394,-22.0,NaN,34900,TMA,52.0,12.5,52,0,58.0
125082,FFF,2014J,269,605394,-22.0,NaN,34901,TMA,94.0,25.0,94,0,43.0


Ой, он не считается!   
Он сдал 1 тестик в одном семестре и потом сдал полностью предмет в другом  
Короче он сдал только 1 семестр, а не два  
Выкидываем его!


### Все теперь давайте посчитаем наконец-то число студентов без экзамена и пересдач, кто сдали 1 курс успешно!

In [44]:
#Выявляем айдишники студентов, у кого сумма весов за тесты == 100, кол-во предметов и семестров == 2
final_students = no_peresdach.query('assessment_type != "Exam" & date_unregistration != date_unregistration & score == score') \
    .groupby('id_student', as_index = False) \
    .agg({'weight' : 'sum','code_module' : pd.Series.nunique, 
          'assessment_type' : 'count','code_presentation' : pd.Series.nunique}) \
    .query('weight == 100 & code_presentation == 2 & code_module == 2').id_student.tolist()

#Создаем новый дата-сет со всеми данными студентов из final_students
final = no_peresdach[no_peresdach.id_student.isin(final_students)]

#Считаем их кол-во успешно сданных тестов с множественными условиями 
#(чтобы кол-во тестов совпадало с тем, что мы выявили для каждого предмета)
final_num = final.query('score >= 40') \
     .groupby(['code_module', 'id_student','code_presentation'], as_index = False) \
     .agg({'assessment_type' : 'count'}) \
     .query('''(code_module == "GGG" & assessment_type >= 9) or \
    (code_module == "BBB" & assessment_type == 4 or assessment_type == 5 or assessment_type == 11) or \
    (code_module == "FFF" & assessment_type > 5)
     ''')

#Здесь мы выявляем дубликаты-студентов
#Потому что нам нужно, чтобы один студент сдал 2 теста в рамках двух различных предметах | семестров
duplicates_num = final_num['id_student'].value_counts()
duplicates_num = duplicates_num[duplicates_num == 2]
duplicates_num = duplicates_num.index.to_list()
final_no_test_students= final_num[final_num.id_student.isin(duplicates_num)].sort_values(by = 'id_student')
final_no_test_students

,code_module,id_student,code_presentation,assessment_type
1,BBB,391937,2014J,5
32,GGG,391937,2014B,9
4,BBB,497074,2014J,5
35,GGG,497074,2013J,9
38,GGG,557085,2013J,9
29,FFF,557085,2014J,12
39,GGG,571334,2013J,9
7,BBB,571334,2014J,5
8,BBB,574512,2014J,4
40,GGG,574512,2013J,9


Считаем количество студентов, кто успешно сдал курс по предметам без экзамена

In [46]:
final_no_test_students.id_student.nunique()

18

### Закончили! 

* Студенты, кто успешно сдал курс (в рамках сдачи 2х экзаменов): 295
* Студенты, кто успешно сдал курс (в рамках сдачи тестов на предмете): 18

Общее количество: 313